```
Yolact_minimal:
```
https://github.com/feiyuhuahuo/Yolact_minimal/tree/master

``` bash
git clone https://github.com/feiyuhuahuo/Yolact_minimal.git
cd Yolact_minimal/
```

In [1]:
#!pip install labelme pycocotools tensorboardX terminaltables

**Train custom datasets (instruction)**

```
1) Use labelme to label your images, only polygons are needed.
1.1 Move images and json-files to './custom_dataset/'
    all_jsons.json, all_imgs.jpg -> /custom_dataset

2) Prepare a 'labels.txt' like this, this first line: 'background' is always needed.
custom_dataset/labels.txt:
        background
        roof

3) Prepare coco-style json, pass the paths of your image folder and the labels.txt. The image type is also needed. The 'custom_dataset' folder is a prepared example.

```

In [2]:
# Prepare coco-style json
#--img_type=jpg
!python utils/labelme2coco.py --img_dir=custom_dataset --label_name=custom_dataset/labels.txt --img_type=png

Created class_name_to_id: {'background': 0, 'roof': 1}.

Generating dataset from: custom_dataset/roof_cam6.json
Generating dataset from: custom_dataset/6.json
Generating dataset from: custom_dataset/roof_cam3.json
Generating dataset from: custom_dataset/custom_ann.json
Traceback (most recent call last):
  File "/usr/src/app/Yolact_minimal/utils/labelme2coco.py", line 35, in <module>
    img_h, img_w = label_data['imageHeight'], label_data['imageWidth']
KeyError: 'imageHeight'


```
4) Edit CUSTOM_CLASSES in config.py. Note that if there's only one class, the CUSTOM_CLASSES should be like ('dog', ). The final comma is necessary to make it as a tuple, or the number of classes would be len('dog').
config.py:
        CUSTOM_CLASSES = ('roof', )
```

**Train**

In [ ]:
# New terminal
# python -m torch.distributed.launch --nproc_per_node=1 --master_port=$((RANDOM)) train.py --cfg=res50_custom

In [1]:
!python -m torch.distributed.launch --nproc_per_node=1 --master_port=$((RANDOM)) train.py --cfg=res50_custom

/opt/conda/lib/python3.10/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
^C



**Test**

In [1]:
# Входные изображения - images_folder/roof
# Результат сохраняется в - results/images
!python detect.py --weight=weights/latest_res50_custom_2000.pth --image=images_folder/roof


------------------------------res50_custom------------------------------
mode: detect
cuda: True
gpu_id: 0
img_size: 544
class_names: ('roof',)
num_classes: 2
scales: [24, 48, 96, 192, 384]
aspect_ratios: [1, 0.5, 2]
weight: weights/latest_res50_custom_2000.pth
traditional_nms: False
nms_score_thre: 0.05
nms_iou_thre: 0.5
top_k: 200
max_detections: 100
image: images_folder/roof
video: None
hide_mask: False
hide_bbox: False
hide_score: False
cutout: False
save_lincomb: False
no_crop: False
real_time: False
visual_thre: 0.3

Model loaded with weights/latest_res50_custom_2000.pth.

Number of all parameters: 30598098

Testing: ████████████████████████████████████████ 2/2, fps: 42.14 | total fps: 5.52 | t_t: 0.181 | t_d: 0.001 | t_f: 0.013 | t_nms: 0.008 | t_after_nms: 0.001 | t_save_img: 0.158
Finished, saved in: results/images.


**export**

In [1]:
!python export2onnx.py --weight='weights/latest_res50_custom_2000.pth' --opset=12


------------------------------res50_custom------------------------------
mode: detect
cuda: True
gpu_id: 0
img_size: 544
class_names: ('roof',)
num_classes: 2
scales: [24, 48, 96, 192, 384]
aspect_ratios: [1, 0.5, 2]
weight: weights/latest_res50_custom_2000.pth
traditional_nms: False
nms_score_thre: 0.05
nms_iou_thre: 0.5
top_k: 200
max_detections: 100
opset: 12

Model loaded with weights/latest_res50_custom_2000.pth.

Number of all parameters: 30598098

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

out: 0, diff: -1.415795850334689e-06
out: 1, diff: -0.021578336134552956
out: 2, diff: -0.0005352501175366342
out: 3, diff: 0.0067505305632948875

Exported as `onnx_files/res50_custom.onnx`.
